In [21]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torchvision.io import read_image
import os

In [64]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_names = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def filename_to_labels(self, filename):
        # Remove the file extension
        name = os.path.splitext(filename)[0]
        # Create a label for each character in the filename
        labels = [f"{char}" for char in name]
        return labels

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = read_image(img_path)

        # Convert the filename to labels
        labels = self.filename_to_labels(self.img_names[idx])

        if self.transform:
            image = self.transform(image)
        return image, labels


In [65]:
dataset = CustomImageDataset('images', transform=None)